# Tarea 2
## IIC2440 - Procesamiento de Datos Masivos

integrantes:
- Rodrigo Nahum
- Fernando Quintana

## Parte 2: Single Source Shortest Path

# Setup

Primero, instalamos pyspark.

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=c03e8169440c2f93dfae41ea19f06183eec8585db4160e96f457c6752af885e2
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


Luego, importamos PySpark y otras librerias a usar, y cramos un Spark Context.

In [2]:
from pyspark.sql import SparkSession
from itertools import permutations
import random
import json
import math

spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext

# Definición del grafo

## Grafo de ejemplo

En esta parte, definimos el grafo a usar. Primero, usamos el grafo dado de ejemplo en la tarea.

In [74]:
node_list = [1, 2, 3, 4]
edges_list = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1), (2, 1, 5), (4, 3, 7)]

nodes = sc.parallelize(node_list)
edges = sc.parallelize(edges_list).map(lambda x: (x[0], (x[1], x[2])))

initial_node = 1

## Grafo aleatorio

Tambien, damos la opción de generar un grafo aleatorio. Abajo, `n` es la cantidad de nodos del grafo, y `edge_chance` es la probabilidad de que una arista cualquiera aparezca en el grafo. O sea, generamos todos los pares de nodos, y para cada uno, decidimos si esa arista está en el grafo, según la probabilidad de `edge_chance`.

In [20]:
n = 10000

edge_chance = 0.002

nodes_list = [i + 1 for i in range(n)]

edges_list = []
for edge in permutations(nodes_list, 2):
    if random.random() < edge_chance:
        edge_cost = random.randint(20, 40)
        edges_list.append([*edge, edge_cost])


nodes = sc.parallelize(nodes_list)
edges = sc.parallelize(edges_list).map(lambda x: (x[0], (x[1], x[2])))

initial_node = 1

## Grafo de Cora

Por último, damos la opción de correr el algoritmo con el grafo de Cora. Esto requiere subir el archivo `cora.zip` al colab.

Primero, hacemos unzip del archivo

In [25]:
!unzip cora.zip

Archive:  cora.zip
   creating: cora/
  inflating: __MACOSX/._cora         
  inflating: cora/cora.content       
  inflating: __MACOSX/cora/._cora.content  
  inflating: cora/README             
  inflating: __MACOSX/cora/._README  
  inflating: cora/cora.cites         
  inflating: __MACOSX/cora/._cora.cites  


Luego, pasamos el grafo al formato requerido por nuestro algoritmo. En este caso, supondremos que el costo de todas las aristas es 1.

In [78]:
nodes_ids = set()
edges_list = []


with open('./cora/cora.cites', 'r') as tf:
    for line in tf:
        edge = line.strip().split('\t')
        first = int(edge[0])
        second = int(edge[1])
        edges_list.append((first, second, 1))
        nodes_ids.add(first)
        nodes_ids.add(second)

nodes = sc.parallelize(list(nodes_ids))
edges = sc.parallelize(edges_list).map(lambda x: (x[0], (x[1], x[2])))

## Nodo inicial elegido al azar
initial_node = 35

# Single Source Shortest Path

## Inicialización

Para implementar este algoritmo, usamos el nodo de inicio definido arriba. Luego, inicializamos los estados de los nodos. En este algoritmo, todos los nodos parten con costo infinito, excepto el nodo inicial, que parte con costo 0.

In [4]:
current_cost = nodes.map(lambda x: (x, 0 if x == initial_node else float('inf')))

## Generación de mensajes.

Luego, debemos crear los mensajes que cada nodo enviará. Para esto, hacemos un join entre los valores actuales y las aristas. Notemos que, en ambos RDDs, la llave es el nodo de origen. De esta forma, obtenemos tuplas que tienen el costo del nodo de origen, y el costo de una arista que sale de ese nodo (además del nodo de destino). Luego, hacemos un map para reordenar la información y sumar el costo del nodo con el de la arista. Obtenemos nuevas tuplas, en donde la llave es el nodo de destino, y el valor es el costo del nodo de origen más el de la arista.

In [5]:
messages = current_cost.join(edges, 4).map(lambda x: (x[1][1][0], (x[1][0] + x[1][1][1])))

## Filtrado de mensajes.

En este caso, sí queremos filtrar mensajes. Los mensajes con costo infinito no nos interesan, ya que no aportan información a los nodos receptores. Entonces, filtramos todos los mensajes con costo infinito.

In [6]:
filtered_messages =  messages.filter(lambda x: x[1] != float('inf'))

## Agregación de mensajes.

Luego, definimos una función para agregar los mensajes recibidos por cada nodo. Para esto, usamos un reduceByKey, de forma de juntar todos los mensajes enviados a un nodo (recordemos que, en los mensajes, la llave es el nodo de destino), y reducirlos según nuestra función de agregación.
En este caso, solo queremos quedarnos con el mínimo costo recibido. Por lo tanto, la función de agregación será simplemente la función `min` de Python.

In [7]:
aggregated_messages = filtered_messages.reduceByKey(lambda x, y: min(x, y))

## Update del estado

Luego, para updatear el estado de los nodos, debemos obtener el estado actual y el resultado de agregar los mensajes. Para esto, usamos un leftOuterJoin entre los estados actuales, y los mensajes agregados (de nuevo, recordemos que la llave de los mensajes agregados es el nodo receptor, entonces, al hacer el leftOuterJoin, tendremos el estado del nodo receptor y el resultado de agregar sus mensajes recibidos). Hacemos un leftOuterJoin, ya que pueden existir nodos sin aristas entrantes, o sea, que no recibieron ningún mensaje.

Luego del leftOuterJoin, hacemos un map para reordenar la información y computar el nuevo valor del nodo. En este caso, de nuevo, nos queremos quedar con el mínimo entre el estado actual y el resultado de agregar los mensajes.

En la función para juntar los valores, se verifica si `tup[1]` es `None` (esto ocurre cuando el nodo no tiene aristas entrantes, o sea, su mensaje agregado es `None`), porque no podemos hacer `min()` con `None`.

In [8]:
def compare_values(tup):
  if (tup[1]) is None:
    return tup[0]
  return min(tup)

new_cost = current_cost.leftOuterJoin(aggregated_messages, 4).map(lambda x: (x[0], compare_values(x[1])))

## Condición de término

Por último, la condición de término es que el valor de ningún nodo haya cambiando entre los valores anteriores y los nuevos valores. Para esto, definimos una nuevoa función que compute esta diferencia. Entonces, hacemos join entre los valores anteriores y los nuevos, obteniendo tuplas que, para cada nodo, tienen el valor anterior y el nuevo valor. Luego, computamos la diferencia entre estos valores, y obtenemos el máximo de todas estas diferencias. Si este máximo es 0, significa que ningún nodo cambió su costo entre la iteración anterior y la nueva.

In [10]:
def get_diff(tup):
  if tup[1][0] == float('inf') and tup[1][1] == float('inf'):
    return 0
  return tup[1][1] - tup[1][0]

max_change = new_cost.join(current_cost, 4).map(get_diff).max()

max_change

inf

## Algoritmo completo

A continuación, mostramos el código completo.

In [79]:
sc.setCheckpointDir('./')

def compare_values(tup):
  if (tup[1]) is None:
    return tup[0]
  return min(tup)

def get_diff(tup):
  if tup[1][0] == float('inf') and tup[1][1] == float('inf'):
    return 0
  return tup[1][1] - tup[1][0]

def single_source_shortest_path(nodes, edges, initial_node):
    current_cost = nodes.map(lambda x: (x, 0 if x == initial_node else float('inf')))


    partitions = math.ceil(nodes.count() / 1000)

    stop = False
    current_cost.checkpoint()

    iter = 0
    while not stop:
        print(f"At iter {iter}")
        messages = current_cost.join(edges, partitions).map(lambda x: (x[1][1][0], (x[1][0] + x[1][1][1])))
        filtered_messages =  messages.filter(lambda x: x[1] != float('inf'))
        new_cost = current_cost.leftOuterJoin(filtered_messages.reduceByKey(lambda x, y: min(x, y)), partitions).map(lambda x: (x[0], compare_values(x[1])))
        new_cost.checkpoint()
        max_change = new_cost.join(current_cost, partitions).map(get_diff).max()
        if max_change == 0:
            stop = True
        else:
            current_cost = new_cost
            iter += 1

    return new_cost.collect()

In [80]:
shortest_paths = single_source_shortest_path(nodes, edges, initial_node)

At iter 0
At iter 1
At iter 2
At iter 3
At iter 4
At iter 5
At iter 6
At iter 7
At iter 8
At iter 9
At iter 10
At iter 11
At iter 12
At iter 13


In [81]:
print(shortest_paths)

[(1105932, 10), (1114125, 4), (253971, inf), (696342, inf), (696345, inf), (16437, inf), (1138755, inf), (1130568, 4), (16461, 5), (16470, inf), (155736, inf), (16476, inf), (1130586, 4), (106590, inf), (589923, 5), (16485, inf), (114, inf), (117, inf), (270456, inf), (409725, inf), (1114239, 5), (1106052, 5), (1130634, 10), (1130637, inf), (180399, 4), (82098, 1), (1130676, inf), (1106103, 10), (1106112, inf), (245955, inf), (590022, 6), (385251, inf), (1106172, 4), (213246, inf), (229635, 1), (270600, inf), (114966, inf), (1138968, 2), (1114398, inf), (213279, inf), (288, inf), (1130808, inf), (49482, 5), (1122642, inf), (1130847, 1), (1130856, 1), (1106298, 2), (24966, inf), (1114512, 3), (147870, inf), (8619, inf), (1130931, 5), (1130934, 9), (1106370, 12), (459213, 2), (459216, 3), (33231, inf), (1106388, 5), (1106406, 3), (1114605, inf), (1106418, inf), (115188, inf), (504, inf), (8703, 7), (1114629, 4), (33303, 3), (573978, 1), (385572, 5), (8766, inf), (746058, inf), (57948, 11

Vemos que (en el caso del grafo de Cora) existen nodos con costo infinito. Esto esta bien, y significa que estos nodos no son alcanzables desde el nodo inicial.